In [13]:
import sklearn
import pandas as pd
import numpy as np
import nltk
import sklearn
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt
from sklearn.model_selection import train_test_split
import re
from nltk.stem import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
import pickle
from wordcloud import WordCloud

In [14]:
full_labels = pd.read_csv('../input_data/full_datasets/fully_labelled_texts.csv')

In [15]:
full_labels

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,Niets gevonden,Niets gevonden
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,Niets gevonden,Niets gevonden
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,Niets gevonden,Niets gevonden
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,Niets gevonden,Niets gevonden
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,Niets gevonden,Niets gevonden
...,...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,Niets gevonden,Niets gevonden
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,Ja,Nee
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,Niets gevonden,Niets gevonden
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,Niets gevonden,Niets gevonden


In [16]:
import pickle
smoking_model = pickle.load(open('../data_exploration/best_models/best_smoking_ex1.pkl', 'rb'))
drinking_model = pickle.load(open('../data_exploration/best_models/best_drinking_ex1.pkl', 'rb'))
drugs_model = pickle.load(open('../data_exploration/best_models/best_drugs_ex1.pkl', 'rb'))

In [17]:
stemmer = SnowballStemmer("dutch")
np.random.seed(500)

def create_preprocess_corpus(input_corpus, content_name, label_name, smoking=False, second_experiment=False, skip_rows=[]):
    corpus = input_corpus[['Unnamed: 0', content_name, label_name]]
    corpus = corpus.rename({content_name:'text', label_name: 'label'}, axis=1)
    corpus['processed_text'] = corpus['text'].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    if second_experiment:
        if smoking:
            replace_text = 'Rookt niet'
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Rookt niet')
            corpus['label'] = corpus['label'].str.replace('Rookte', 'Rookt niet')
        else:
            corpus['label'] = corpus['label'].str.replace('Niets gevonden', 'Nee')
        
    corpus = corpus.drop(corpus[corpus.label == '--'].index)
    corpus = corpus.drop(corpus[corpus.label == 'Onbekend'].index)
    corpus = corpus[~corpus.processed_text.str.contains('vertrouwelijk')]
    corpus_backup = corpus.copy()
    return corpus, corpus_backup

def add_processed_text(input_corpus, text_column):
    corpus = input_corpus.copy()
    corpus['processed_text'] = corpus[text_column].str.replace('\t',' ')
    corpus.drop_duplicates(inplace=True)
    corpus['processed_text'] = corpus['processed_text'].astype(str)
    corpus['processed_text'] = corpus['processed_text'].str.lower()
    corpus['processed_text'] = [stemmer.stem(text) for text in corpus['processed_text']]
    return corpus

In [18]:
smoking_corpus, _ = create_preprocess_corpus(full_labels, 'content', 'roken_answer_label', smoking=True)
drinking_corpus, _ = create_preprocess_corpus(full_labels, 'content', 'alcohol_answer_label')
drugs_corpus, _ = create_preprocess_corpus(full_labels, 'content', 'drugs_answer_label')

In [19]:
smoking_corpus

,Unnamed: 0,text,label,processed_text
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,"indicatie: x-bekken, x-heup axiaal klinishe ..."
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,"anamnese: geen klachten van pijn op de borst,..."
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,   [initials] [lastname] [streetname] e...
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,"geachte collega, bovengenoemde patient was op..."
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,"geachte collega, bovengenoemde patiente was o..."
...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,beloop: beloop: tc op verzoek patient beleid...
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Rookte,anamnese: verkort consult: voorgeschiedenis ...
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,beloop: beloop: zie mri lang traject 7 cm ext...
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,samenvatting: voorgeschiedenis: blanco. medic...


In [20]:



drinking_corpus

,Unnamed: 0,text,label,processed_text
0,0,"Indicatie:\tX-bekken, x-heup axiaal \t\tKlinis...",Niets gevonden,"indicatie: x-bekken, x-heup axiaal klinishe ..."
1,5,Anamnese: \tGeen klachten van pijn op de borst...,Niets gevonden,"anamnese: geen klachten van pijn op de borst,..."
2,6, \t\t\t\t\t\t[INITIALS] [LASTNAME] [STREETN...,Niets gevonden,   [initials] [lastname] [streetname] e...
3,10,"Geachte collega,\t\tBovengenoemde patiënt was ...",Niets gevonden,"geachte collega, bovengenoemde patient was op..."
4,12,"Geachte collega,\t\tBovengenoemde patiënte was...",Niets gevonden,"geachte collega, bovengenoemde patiente was o..."
...,...,...,...,...
59798,652070,Beloop: \tBeloop: TC op verzoek patient\t\tBel...,Niets gevonden,beloop: beloop: tc op verzoek patient beleid...
59799,652075,Anamnese: \t Verkort consult: Voorgeschiedenis...,Ja,anamnese: verkort consult: voorgeschiedenis ...
59800,652078,Beloop: \tBeloop: zie MRI\tlang traject 7 cm e...,Niets gevonden,beloop: beloop: zie mri lang traject 7 cm ext...
59801,652082,Samenvatting: \tVoorgeschiedenis: blanco.\tMed...,Niets gevonden,samenvatting: voorgeschiedenis: blanco. medic...


In [21]:
_, smoking_x_test, _, smoking_y_test = train_test_split(smoking_corpus['processed_text'], smoking_corpus['label'], test_size=0.2, random_state=50)
_, drinking_x_test, _, drinking_y_test = train_test_split(drinking_corpus['processed_text'], drinking_corpus['label'], test_size=0.2, random_state=50)
_, drugs_x_test, _, drugs_y_test = train_test_split(drugs_corpus['processed_text'], drugs_corpus['label'], test_size=0.2, random_state=50)

In [24]:
part1 = pd.read_csv("full_texts_to_label_full4.csv")
part2 = pd.read_csv("full_texts_to_label_random1.csv")
part3 = pd.read_csv("full_texts_to_label_random2.csv")
part4 = pd.read_csv("full_texts_to_label_random3.csv")
full_label_texts = pd.concat([part1, part2, part3, part4])
full_label_processed = add_processed_text(full_label_texts, "text")

In [25]:
full_label_processed

,Unnamed: 0,text,roken_answer_label,alcohol_answer_label,drugs_answer_label,processed_text
0,0,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Nee,Nee,anamnese: verkort consult: aandachtspunten u...
1,1,Anamnese: \t Verkort consult: 28-2-2022 Rad TC...,Rookte,Ja,Nee,anamnese: verkort consult: 28-2-2022 rad tc ...
2,2,Anamnese: \tRvK: pijn op de borst en palpitati...,Rookt,Ja,Nee,anamnese: rvk: pijn op de borst en palpitatie...
3,3,"Samenvatting: \tglaucoom ods, wv gtt \t2014 p...",Niets gevonden,Niets gevonden,Niets gevonden,"samenvatting: glaucoom ods, wv gtt 2014 pse..."
4,4,Samenvatting: \t2/1/23 vitrectomie + Avastin +...,Niets gevonden,Niets gevonden,Niets gevonden,samenvatting: 2/1/23 vitrectomie + avastin + ...
...,...,...,...,...,...,...
496,31188,Beloop: \tBeloop: Huisarts van HAP belde.\tSte...,Niets gevonden,Niets gevonden,Niets gevonden,beloop: beloop: huisarts van hap belde. sterk...
497,59730,UITSLAG MICROBIOLOGIE\t\t[INITIALS] [LASTNAME]...,Niets gevonden,Niets gevonden,Niets gevonden,uitslag microbiologie [initials] [lastname] d...
498,32962,UITSLAG MICROBIOLOGIE\t\t[INITIALS] [LASTNAME]...,Niets gevonden,Niets gevonden,Niets gevonden,uitslag microbiologie [initials] [lastname] ...
499,25760,Anamnese: \t Verkort consult: Aandachtspunten ...,Rookte,Nee,Nee,anamnese: verkort consult: aandachtspunten u...


# Smoking

In [26]:
predicted_nb_df = smoking_model.decision_function(smoking_x_test)
predicted_nb_labels = smoking_model.predict(smoking_x_test)
x_test_probs = pd.DataFrame(smoking_x_test, columns=['processed_text'])
x_test_probs['nothing_found_eval'] = [x[0] for x in predicted_nb_df]
x_test_probs['current_user_eval'] = [x[1] for x in predicted_nb_df]
x_test_probs['non_user_eval'] = [x[2] for x in predicted_nb_df]
x_test_probs['previous_user_eval'] = [x[3] for x in predicted_nb_df]
x_test_probs['predicted_label'] = predicted_nb_labels
x_test_probs['roken_answer_label'] = smoking_y_test
x_test_probs = x_test_probs.merge(smoking_corpus[['text', 'processed_text']], left_on='processed_text', right_on='processed_text', how='left')

In [27]:
sorted_probs_non = x_test_probs[~x_test_probs.processed_text.isin(full_label_processed.processed_text)].sort_values(by=['non_user_eval'], ascending=False)
texts_to_label = sorted_probs_non[['text', 'processed_text', 'roken_answer_label']].head(100)
#texts_to_label = texts_to_label.append(sorted_probs_previous[~sorted_probs_previous.processed_text.isin(texts_to_label.processed_text)][['text', 'processed_text', 'roken_answer_label']].head(67))
texts_to_label = texts_to_label.merge(drinking_corpus.rename(columns={'label': 'alcohol_answer_label'})[['processed_text', 'alcohol_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label = texts_to_label.merge(drugs_corpus.rename(columns={'label': 'drugs_answer_label'})[['processed_text', 'drugs_answer_label']], left_on='processed_text', right_on='processed_text', how='left')
texts_to_label = texts_to_label.drop_duplicates()

In [28]:
texts_to_label

,text,processed_text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Intoxicaties: \tRoken- Roken: ja; Alcohol- Alc...,intoxicaties: roken- roken: ja; alcohol- alco...,Rookt niet,Nee,Nee
1,"Anamnese: \tIntoxicaties: rookt niet, drinkt n...","anamnese: intoxicaties: rookt niet, drinkt ni...",Rookt niet,Niets gevonden,Niets gevonden
2,Samenvatting: \tVoorgeschiedenis: OSAS AHI=26\...,samenvatting: voorgeschiedenis: osas ahi=26 m...,Rookt niet,Niets gevonden,Niets gevonden
3,Samenvatting: \tVoorgeschiedenis: gb\tMedicati...,samenvatting: voorgeschiedenis: gb medicatie:...,Niets gevonden,Niets gevonden,Niets gevonden
4,Samenvatting: \tVoorgeschiedenis: hooikoorts\t...,samenvatting: voorgeschiedenis: hooikoorts me...,Rookt niet,Niets gevonden,Niets gevonden
...,...,...,...,...,...
95,Beloop: \tCONSULTENKAMERGezien door co-ass Y. ...,beloop: consultenkamergezien door co-ass y. t...,Rookt niet,Nee,Nee
96,Reden van komst / Verwijzing: \tReden verwijzi...,reden van komst / verwijzing: reden verwijzin...,Rookt niet,Nee,Nee
97,"Beloop: \tMR CPG7P4M4 // MI: 4x sectio ia, 1x...","beloop: mr cpg7p4m4 // mi: 4x sectio ia, 1x ...",Rookt niet,Nee,Nee
98,Reden van komst / Verwijzing: \tReden van koms...,reden van komst / verwijzing: reden van komst...,Rookt niet,Nee,Nee


In [29]:
full_texts_to_label = texts_to_label.drop(['processed_text'], axis=1)

In [30]:
full_texts_to_label

,text,roken_answer_label,alcohol_answer_label,drugs_answer_label
0,Intoxicaties: \tRoken- Roken: ja; Alcohol- Alc...,Rookt niet,Nee,Nee
1,"Anamnese: \tIntoxicaties: rookt niet, drinkt n...",Rookt niet,Niets gevonden,Niets gevonden
2,Samenvatting: \tVoorgeschiedenis: OSAS AHI=26\...,Rookt niet,Niets gevonden,Niets gevonden
3,Samenvatting: \tVoorgeschiedenis: gb\tMedicati...,Niets gevonden,Niets gevonden,Niets gevonden
4,Samenvatting: \tVoorgeschiedenis: hooikoorts\t...,Rookt niet,Niets gevonden,Niets gevonden
...,...,...,...,...
95,Beloop: \tCONSULTENKAMERGezien door co-ass Y. ...,Rookt niet,Nee,Nee
96,Reden van komst / Verwijzing: \tReden verwijzi...,Rookt niet,Nee,Nee
97,"Beloop: \tMR CPG7P4M4 // MI: 4x sectio ia, 1x...",Rookt niet,Nee,Nee
98,Reden van komst / Verwijzing: \tReden van koms...,Rookt niet,Nee,Nee


In [31]:
full_texts_to_label.to_csv('full_texts_to_label_full_previous.csv')

In [30]:
full_labels_no_dup.sample(frac=1).head(1)

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
56849,644975,"Samenvatting: \tElectief, 3VD \t\t06-05-2022 C...",Niets gevonden,Niets gevonden,Niets gevonden


In [31]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

In [32]:
full_labels_no_dup.sample(frac=1).head(1)

,Unnamed: 0,content,roken_answer_label,alcohol_answer_label,drugs_answer_label
47608,619040,"Samenvatting: \t06-2021 pterygium excisie OD\t\t\tAnamnese: \tgaat goed.\t\thylan zn \t\t\tLichamelijk onderzoek: \tLichamelijk onderzoekOogdruk OD: 18 mmHg, applanatie. Oogdruk OS: 18 mmHg, applanatie. Spleetlamp OD: inferior punctata, voorste oogkamer matig diepgonio tm 3-4 kw a vue. Spleetlamp OS: geen pterygium, voorste oogkamer matig diepgonio tm 3-4 kw a vue. Fundoscopie ODS: scherp begrensde papillen, niet suspect. \t\tConclusie: \tConclusie: iets sicca nog bij traanfilm insufficiëntie \t\tBeleid: \tBeleid: herstart vidisic \thylan door \t\t\tOverige acties: \tAfspraak maken: Controle afspraak over 1j. Opmerking: CPTOA Steen .",Niets gevonden,Niets gevonden,Niets gevonden
